In [88]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interactive_output, FloatSlider, Dropdown, HTML, Layout, Output, HBox, VBox
from IPython.display import display
from sklearn.linear_model import LinearRegression

In [2]:
# Load data
full_data = pd.read_csv("Constant Strain Rate Data.csv")
epsT_full = np.array(full_data["Strain"], dtype=float)
SR = 1.0  # set strain rate (you can add a slider for this if desired)
temps = [c for c in full_data.columns if c != "Strain"]

param_names = ["E", "k", "K", "n1", "A", "C", "n2", "B"]
temps_sorted = np.array(sorted([float(t) for t in temps]))
params_by_temp = {
    "200": dict(E=30000.0, k=50.0, K=200.0, n1=1.0, A=5.0, C=0.5, n2=1.5, B=200.0),
    "250": dict(E=28000.0, k=45.0, K=210.0, n1=1.0, A=4.5, C=0.6, n2=1.5, B=190.0),
    "300": dict(E=26000.0, k=40.0, K=220.0, n1=1.0, A=4.0, C=0.7, n2=1.5, B=180.0),
    "350": dict(E=26000.0, k=40.0, K=220.0, n1=1.0, A=4.0, C=0.7, n2=1.5, B=180.0),
    "400": dict(E=26000.0, k=40.0, K=220.0, n1=1.0, A=4.0, C=0.7, n2=1.5, B=180.0),
    "450": dict(E=26000.0, k=40.0, K=220.0, n1=1.0, A=4.0, C=0.7, n2=1.5, B=180.0),
    "500": dict(E=26000.0, k=40.0, K=220.0, n1=1.0, A=4.0, C=0.7, n2=1.5, B=180.0),
    "535": dict(E=26000.0, k=40.0, K=220.0, n1=1.0, A=4.0, C=0.7, n2=1.5, B=180.0)
}

print(temps_sorted)

[200. 250. 300. 350. 400. 450. 500. 535.]


In [3]:
# numerical scheme 
def simulate_sigma_euler(
    epsT: np.ndarray,          # true strain array (monotonic increasing)
    SR: float,                 # strain rate (1/s)
    E: float, k: float, K: float, n1: float,
    A: float, C: float, n2: float,
    B: float,
    eps_p0: float = 0.0,
    rho0: float = 1e-4,
    rho_floor: float = 1e-12,
    n_sub: int = 1,            # set to e.g. 10, 50 if it jumps
):
    """
    Forward Euler on eps_p and rho, with algebraic R = B*sqrt(rho).
    Returns sigma_model, eps_p_hist, rho_hist, R_hist
    All stress-like quantities must be in consistent units (e.g., MPa everywhere).
    """

    # ensure epsT is np.array
    epsT = np.asarray(epsT, dtype=float)
    # number of sigma values to generate
    N = len(epsT)

    sigma = np.zeros(N)
    eps_p = np.zeros(N)
    rho = np.zeros(N)
    R = np.zeros(N)

    # intialise plastic strain as zero
    eps_p[0] = eps_p0

    # why is rho bounded?
    rho[0] = max(rho0, rho_floor)
    R[0] = B * np.sqrt(rho[0])
    sigma[0] = E * (epsT[0] - eps_p[0])

    # sub-stepping routine for stability 
    for i in range(1, N):
        # compute strain step size
        d_epsT = epsT[i] - epsT[i-1]
        if d_epsT <= 0:
            raise ValueError("epsT must be strictly increasing.")
        # compute time step size
        dt = d_epsT / SR

        # substepping (optional but very effective)
        dt_sub = dt / n_sub
        epsT_sub0 = epsT[i-1]

        epp = eps_p[i-1]
        r = rho[i-1]
        
        # if we have instability we can reduce the time step
        for s in range(n_sub):
            # linearl
            epsT_s = epsT_sub0 + (s + 1) * (d_epsT / n_sub)

            r = max(r, rho_floor)
            R_s = B * np.sqrt(r)
            sigma_s = E * (epsT_s - epp)

            drive = (sigma_s - R_s - k) / K
            epp_dot = max(drive, 0) ** n1

            r_dot = A * (1.0 - r) * epp_dot - C * (r ** n2)

            # Euler update
            epp = epp + epp_dot * dt_sub
            r = r + r_dot * dt_sub

            r = max(r, rho_floor)

        eps_p[i] = epp
        rho[i] = r
        R[i] = B * np.sqrt(rho[i])
        sigma[i] = E * (epsT[i] - eps_p[i])

    return sigma, eps_p, rho, R


In [23]:
# linear regression for fitting
def linear_regression(x, y):
    model = LinearRegression()
    model.fit(x, y)
    intercept = model.intercept_
    grad = model.coef_
    intercept = np.ravel(intercept).item()
    grad = np.ravel(grad).item()
    return float(intercept), float(grad)

In [100]:
# Create an Output widget for the plot
output_model = Output()
output_params = Output()

# Interactive plotting function
def plot_simulation(temp, E, k, K, n1, A, C, n2, B):
    with output_model:
        output_model.clear_output(wait=True)  # Clear previous plot
        sigma_exp = full_data[temp].to_numpy()
        mask = ~np.isnan(sigma_exp)
        epsT = epsT_full[mask]
        sigma_exp = sigma_exp[mask]
        
        sigma_model, _, _, _ = simulate_sigma_euler(
            epsT=epsT,
            SR=SR,
            E=E, k=k, K=K, n1=n1, A=A, C=C, n2=n2, B=B,
            rho0=1e-4,
            n_sub=1
        )
        
        plt.figure(figsize=(5, 3))
        plt.title(f"Simulation and experiment plot for {temp}°C", fontsize=10)
        plt.plot(epsT, sigma_model, label='Model', color='blue')
        plt.scatter(epsT, sigma_exp, label='Experiment', color='red', alpha=0.7)
        plt.xlabel('Strain', fontsize = 7)
        plt.ylabel('Stress (MPa)', fontsize = 7)
        plt.legend()
        plt.grid(True)
        plt.show()

def plot_regression(temp, E, k, K, n1, A, C, n2, B):
    with output_params:
        output_params.clear_output(wait=True)
        
        
        params_by_temp[temp] = {
            "E": E, "k": k, "K": K, "n1": n1,
            "A": A, "C": C, "n2": n2, "B": B
        }

        # param vs temperature plots
        fig, axes = plt.subplots(2, 4, figsize=(20, 8), sharex=True)
        axes = axes.ravel()
        for ax, name in zip(axes, param_names):
            # select the variable of interest
            y = np.array([params_by_temp[temp][name] for temp in temps])
            # fitting in logspace against, 1/T (Kelvin)
            log_y = np.log(y).reshape(-1, 1)
            T_inv = (1 / (temps_sorted + 273.15)).reshape(-1,1)
            intercept, grad = linear_regression(T_inv, log_y)
            fit = lambda x : grad * x + intercept
            ax.plot(T_inv, fit(T_inv), linestyle="--")
            ax.scatter(T_inv, log_y, c='r', marker="x")
            ax.text(0.2, 0.85, 
                    f"log({name}) = {intercept:.2f} + {grad:.2f} (1 / T)", 
                    fontsize = 15,
                    transform=ax.transAxes)
            ax.tick_params(labelbottom=True)
            ax.set_title(f"log({name}) vs 1 / T ")
            ax.set_xlabel("1 / K", fontsize = 7)
            ax.set_ylabel(f"log({name})", fontsize = 7)
            ax.grid(True)

        plt.tight_layout()
        plt.show()

def full_plot(temp, E, k, K, n1, A, C, n2, B):
    plot_simulation(temp, E, k, K, n1, A, C, n2, B)
    plot_regression(temp, E, k, K, n1, A, C, n2, B)

# Create interactive widgets
temp_dropdown = Dropdown(options=temps, value=temps[0], description='Temp:', layout=Layout(width='200px', height='20px'))
E_slider = FloatSlider(min=1000, max=80000, step=500, value=30000, description='E (MPa):', layout=Layout(width='500px', height='10px'))
k_slider = FloatSlider(min=0, max=200, step=1, value=50, description='k (MPa):', layout=Layout(width='500px', height='10px'))
K_slider = FloatSlider(min=10, max=500, step=1, value=200, description='K (MPa):', layout=Layout(width='500px', height='10px'))
n1_slider = FloatSlider(min=0.01, max=2, step=0.05, value=1.0, description='n1:', layout=Layout(width='500px', height='10px'))
A_slider = FloatSlider(min=0.01, max=5, step=0.05, value=5.0, description='A:', layout=Layout(width='500px', height='10px'))
C_slider = FloatSlider(min=1, max=200, step=1, value=0.5, description='C:', layout=Layout(width='500px', height='10px'))
n2_slider = FloatSlider(min=0.1, max=5, step=0.1, value=1.5, description='n2:', layout=Layout(width='500px', height='10px'))
B_slider = FloatSlider(min=10, max=500, step=1, value=200, description='B (MPa):', layout=Layout(width='500px', height='10px'))

# format layout, sliders in a vertical box
controls = VBox([
    temp_dropdown,
    E_slider, k_slider, K_slider, n1_slider,
    A_slider, C_slider, n2_slider, B_slider
], layout=Layout(width="700px"))

# group outputs
output_model.layout  = Layout(width="700px", height="500px")
output_params.layout = Layout(width="1000px", height="500px")
spacer = HTML(value="", layout=Layout(height="20px"))

plots = HBox([output_model, spacer, controls])

display(VBox([output_params, plots]))

# Use interact to create the interactive plot
full_display = interactive_output(full_plot,
            {"temp": temp_dropdown,
            "E":E_slider, 
            "k":k_slider, 
            "K":K_slider, 
            "n1":n1_slider, 
            "A":A_slider, 
            "C":C_slider, 
            "n2":n2_slider, 
            "B":B_slider})

display(full_display)

Output()

In [84]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Output, interact, Dropdown, FloatSlider, Layout
from sklearn.linear_model import LinearRegression

# One output widget
output_all = Output()

def linear_regression(x, y):
    """
    x: (N,) or (N,1)
    y: (N,) or (N,1)
    returns: (intercept: float, grad: float)
    """
    x = np.asarray(x).reshape(-1, 1)
    y = np.asarray(y).reshape(-1, 1)

    model = LinearRegression()
    model.fit(x, y)

    intercept = float(np.ravel(model.intercept_)[0])
    grad = float(np.ravel(model.coef_)[0])
    return intercept, grad


def full_plot(temp, E, k, K, n1, A, C, n2, B):
    with output_all:
        output_all.clear_output(wait=True)

        # --- update params store ---
        params_by_temp[temp] = {"E": E, "k": k, "K": K, "n1": n1, "A": A, "C": C, "n2": n2, "B": B}

        # --- prepare data for model plot ---
        sigma_exp = full_data[temp].to_numpy()
        mask = ~np.isnan(sigma_exp)
        epsT = epsT_full[mask]
        sigma_exp = sigma_exp[mask]

        sigma_model, _, _, _ = simulate_sigma_euler(
            epsT=epsT,
            SR=SR,
            E=E, k=k, K=K, n1=n1, A=A, C=C, n2=n2, B=B,
            rho0=1e-4,
            n_sub=1
        )

        # --- build one combined figure ---
        fig = plt.figure(figsize=(14, 7), constrained_layout=True)
        gs = fig.add_gridspec(1, 2, width_ratios=[1.0, 1.6])

        # Left: model vs experiment
        ax0 = fig.add_subplot(gs[0, 0])
        ax0.set_title(f"Simulation vs experiment ({temp}°C)", fontsize=11)
        ax0.plot(epsT, sigma_model, label="Model")
        ax0.scatter(epsT, sigma_exp, label="Experiment", alpha=0.7, s=12)
        ax0.set_xlabel("Strain")
        ax0.set_ylabel("Stress (MPa)")
        ax0.grid(True)
        ax0.legend()

        # Right: 4x2 regression grid (subgridspec)
        sub = gs[0, 1].subgridspec(4, 2, wspace=0.25, hspace=0.35)
        axes = [fig.add_subplot(sub[i, j]) for i in range(4) for j in range(2)]

        # shared x for regression plots (manually) + show labels everywhere
        # Make 1D arrays for plotting
        T_inv_1d = 1.0 / (temps_sorted + 273.15)  # (N,)
        for ax, name in zip(axes, param_names):
            y = np.array([params_by_temp[t][name] for t in temps])  # (N,)
            log_y_1d = np.log(y)  # (N,)

            intercept, grad = linear_regression(T_inv_1d, log_y_1d)

            fit_y = grad * T_inv_1d + intercept

            ax.plot(T_inv_1d, fit_y, linestyle="--")
            ax.scatter(T_inv_1d, log_y_1d, marker="x")

            ax.text(
                0.03, 0.97,
                f"ln({name}) = {intercept:.2f} + {grad:.2f} (1/T)",
                transform=ax.transAxes,
                va="top", ha="left",
                fontsize=9,
                bbox=dict(boxstyle="round", facecolor="white", alpha=0.7, linewidth=0.5),
            )

            ax.set_title(f"{name} vs 1/T", fontsize=10)
            ax.set_xlabel("1/T (1/K)", fontsize=8)
            ax.set_ylabel(f"ln({name})", fontsize=8)   # <- this matches what you're plotting
            ax.tick_params(labelbottom=True)
            ax.grid(True)

        plt.show()


# Display ONE widget (everything inside it)
display(output_all)

# interact as before
interact(
    full_plot,
    temp=temp_dropdown,
    E=E_slider,
    k=k_slider,
    K=K_slider,
    n1=n1_slider,
    A=A_slider,
    C=C_slider,
    n2=n2_slider,
    B=B_slider,
)


Output()

interactive(children=(Dropdown(description='Temp:', layout=Layout(height='20px', width='200px'), options=('200…

<function __main__.full_plot(temp, E, k, K, n1, A, C, n2, B)>